# AOC2022

## Day 22 / Part 1 / Monkey Map

Problem Description: https://adventofcode.com/2022/day/22

Input: [Example](aoc2022_day22_example.txt)

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
"""Solution for AOC2022, day 22, part 1."""
# required for forward references (causing E0602)
from __future__ import annotations

from dataclasses import dataclass
import enum
import logging
import re
import sys
import numpy as np

LOGGER = logging.getLogger(__name__)

# show/hide debug logs
SHOW_DEBUG_LOG = False
# set input file
INPUT_FILE = "aoc2022_day22_example.txt"

In [3]:
class Direction(enum.IntEnum):
    """Direction enumeration."""
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3


@dataclass
class Tile:
    """Class representing a tile."""
    pos: list[int]
    up_tile: Tile = None
    right_tile: Tile = None
    down_tile: Tile = None
    left_tile: Tile = None


class Grid:
    """Grid of linked tiles."""
    board_map = None
    tiles = None

    def __init__(self, board_map):
        self.board_map = board_map
        self._build()

    def _compute_up_tile_pos(self, y_pos, x_pos):
        """
        Internal method for computing the upper neighbour of a tile.
        """
        up_tile_pos = None
        if y_pos-1 < 0 or self.board_map[y_pos-1, x_pos] == " ":
            y_delta = self.board_map.shape[0]-1
            while self.board_map[y_delta, x_pos] == " ":
                y_delta -= 1
            up_tile_pos = (y_delta, x_pos)
        else:
            up_tile_pos = (y_pos-1, x_pos)
        if self.board_map[up_tile_pos[0], up_tile_pos[1]] == "#":
            up_tile_pos = None
        return up_tile_pos

    def _compute_right_tile_pos(self, y_pos, x_pos):
        """
        Internal method for computing the right neighbour of a tile.
        """
        right_tile_pos = None
        if (
            x_pos+1 >= self.board_map.shape[1] or
            self.board_map[y_pos, x_pos+1] == " "
        ):
            x_delta = 0
            while self.board_map[y_pos, x_delta] == " ":
                x_delta += 1
            right_tile_pos = (y_pos, x_delta)
        else:
            right_tile_pos = (y_pos, x_pos+1)
        if self.board_map[right_tile_pos[0], right_tile_pos[1]] == "#":
            right_tile_pos = None
        return right_tile_pos

    def _compute_down_tile_pos(self, y_pos, x_pos):
        """
        Internal method for computing the below neighbour of a tile.
        """
        down_tile_pos = None
        if (
            y_pos+1 >= self.board_map.shape[0] or
            self.board_map[y_pos+1, x_pos] == " "
        ):
            y_delta = 0
            while self.board_map[y_delta, x_pos] == " ":
                y_delta += 1
            down_tile_pos = (y_delta, x_pos)
        else:
            down_tile_pos = (y_pos+1, x_pos)
        if self.board_map[down_tile_pos[0], down_tile_pos[1]] == "#":
            down_tile_pos = None
        return down_tile_pos

    def _compute_left_tile_pos(self, y_pos, x_pos):
        """
        Internal method for computing the left neighbour of a tile.
        """
        left_tile_pos = None
        if x_pos-1 < 0 or self.board_map[y_pos, x_pos-1] == " ":
            x_delta = self.board_map.shape[1]-1
            while self.board_map[y_pos, x_delta] == " ":
                x_delta -= 1
            left_tile_pos = (y_pos, x_delta)
        else:
            left_tile_pos = (y_pos, x_pos-1)
        if self.board_map[left_tile_pos[0], left_tile_pos[1]] == "#":
            left_tile_pos = None
        return left_tile_pos

    def _build(self):
        """Internal method for building the grid."""
        self.tiles = {}
        for y_pos in range(self.board_map.shape[0]):
            for x_pos in range(self.board_map.shape[1]):
                if self.board_map[y_pos, x_pos] != ".":
                    continue

                if (y_pos, x_pos) not in self.tiles:
                    self.tiles[(y_pos, x_pos)] = Tile((y_pos, x_pos))

                # 1. compute neighbor tile positions
                up_tile_pos = self._compute_up_tile_pos(y_pos, x_pos)
                right_tile_pos = self._compute_right_tile_pos(y_pos, x_pos)
                down_tile_pos = self._compute_down_tile_pos(y_pos, x_pos)
                left_tile_pos = self._compute_left_tile_pos(y_pos, x_pos)

                # 2. link tiles
                for nid, neigh_tile_pos in enumerate(
                    [up_tile_pos, right_tile_pos, down_tile_pos, left_tile_pos]
                ):
                    if neigh_tile_pos is not None:
                        neigh_tile = None
                        if neigh_tile_pos in self.tiles:
                            neigh_tile = self.tiles[neigh_tile_pos]
                        else:
                            neigh_tile = Tile(neigh_tile_pos)
                            self.tiles[neigh_tile_pos] = neigh_tile

                        if nid == 0:
                            self.tiles[(y_pos, x_pos)].up_tile = neigh_tile
                        if nid == 1:
                            self.tiles[(y_pos, x_pos)].right_tile = neigh_tile
                        if nid == 2:
                            self.tiles[(y_pos, x_pos)].down_tile = neigh_tile
                        if nid == 3:
                            self.tiles[(y_pos, x_pos)].left_tile = neigh_tile

    def __str__(self):
        """Compute the informal string representation of the node."""
        return self.board_map

    def __getitem__(self, idx):
        """Get tile at index idx."""
        return self.tiles[idx]


class GridWalker:
    """Helper class for walking on a grid of tiles."""
    curr_tile = None
    direction = 0

    def __init__(self, start_tile, direction=Direction.RIGHT):
        self.curr_tile = start_tile
        self.direction = direction

    def turn_left(self):
        """Change facing direction to the left."""
        self.direction = (self.direction - 1) % 4

    def turn_right(self):
        """Change facing direction to the right."""
        self.direction = (self.direction + 1) % 4

    def move(self, steps):
        """Move forward an amount of steps."""
        for _ in range(steps):
            if (
                self.direction == Direction.UP and
                self.curr_tile.up_tile is not None
            ):
                self.curr_tile = self.curr_tile.up_tile
            if (
                self.direction == Direction.RIGHT and
                self.curr_tile.right_tile is not None
            ):
                self.curr_tile = self.curr_tile.right_tile
            if (
                self.direction == Direction.DOWN and
                self.curr_tile.down_tile is not None
            ):
                self.curr_tile = self.curr_tile.down_tile
            if (
                self.direction == Direction.LEFT and
                self.curr_tile.left_tile is not None
            ):
                self.curr_tile = self.curr_tile.left_tile

In [4]:
def find_start_pos(board_map):
    """Method to find the start position of a board_map."""
    start_pos = None
    first_row = board_map[0]
    for col_idx, cell in enumerate(first_row):
        if cell == ".":
            start_pos = (0, col_idx)
            break
    return start_pos

In [5]:
def main():
    """Main function to solve puzzle."""
    board_map = []
    board_map_width = 0
    path = []
    with open(INPUT_FILE, encoding="utf-8") as file_obj:
        read_mode = "map"
        for line in [line.rstrip() for line in file_obj.readlines()]:
            if line == "":
                read_mode = "path"
                continue
            if read_mode == "map":
                board_map.append(list(line))
                if len(line) > board_map_width:
                    board_map_width = len(line)
            if read_mode == "path":
                path = line

    # make all rows the same width
    for row in board_map:
        row.extend(
            list(" " * (board_map_width - len(row)))
        )

    start_pos = find_start_pos(board_map)

    board_map = np.asarray(board_map)

    LOGGER.debug(
        "board map:\n  %s\n",
        "\n  ".join(["".join(row) for row in board_map])
    )

    grid = Grid(board_map)
    grid_walker = GridWalker(grid[start_pos])

    LOGGER.debug("move along grid...")
    LOGGER.debug("  traverse tile %s", start_pos)
    for steps, turn in re.findall(r"(\d+)([RL])", path):
        grid_walker.move(int(steps))
        if turn == "L":
            grid_walker.turn_left()
        else:
            grid_walker.turn_right()
        LOGGER.debug("  traverse tile %s", grid_walker.curr_tile.pos)

    LOGGER.debug("")

    row, col, facing = (
        grid_walker.curr_tile.pos[0], grid_walker.curr_tile.pos[1],
        grid_walker.direction
    )

    print(f"solution: {(row+1) * 1000 + (col+1) * 4 + ((facing-1) % 4)}")

In [6]:
if __name__ == "__main__":
    LOGGER.setLevel(logging.DEBUG if SHOW_DEBUG_LOG else logging.INFO)
    log_formatter = logging.Formatter("%(message)s")
    log_handler = logging.StreamHandler(sys.stdout)
    log_handler.setFormatter(log_formatter)
    LOGGER.addHandler(log_handler)
    main()

solution: 6032
